# Code for generating simulation input files

By Jessica H. Sun and Abigail Plummer

Licensed under the terms of GNU GENERAL PUBLIC LICENSE by Free Software Foundation

Helper functions for generating run_dir_list.ipynb which contains a list of subfolder names that contain experiment parameters to run

We already provided the input files, but you can use the following to generate arbitrary experiment parameter inputs.

---
1. Generate run_dir_list.npy

2. Edit existing run_dir_list.npy

3. Generate run_dir_list.npy for commensurate cylinders and cylinder_params_df.pkl
---

1. Generate run_dir_list.npy

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def ctheta_to_alpha(cthetadeg): #ctheta is sector angle. alpha is half opening cone angle
    # alpha=theta/2=arcsin(phi/(2pi)), theta=full cone angle, alpha=half cone angle, phi=sector angle
    cthetarad=cthetadeg*np.pi/180
    alpharad=np.arcsin(cthetarad/(2*np.pi))
    alphadeg=alpharad*180/np.pi
    return alphadeg
def alpha_to_ctheta(alphadeg):
    alpharad=alphadeg*np.pi/180
    cthetarad=np.sin(alpharad)*(2*np.pi)
    cthetadeg=cthetarad*180/np.pi
    return cthetadeg
def magic_cinit_list(cinit_min=5,cinit_max=12):
    #for magic cylinders
    magic_cinit_list=np.arange(0,cinit_max+10,np.sqrt(3))
    magic_cinit_list=np.append(magic_cinit_list,np.arange(0,cinit_max+10,1))
    magic_cinit_list=np.unique(magic_cinit_list)

    #for magic cones - note that yinit changes with cone angle (i.e. low cone angles will initialize at very large yinit for a fixed circumference)
    magic_yinit_list=np.arange(0,cinit_max+10,np.sqrt(3)/2)
    magic_yinit_list=np.append(magic_yinit_list,np.arange(0,cinit_max+10,1))
    magic_yinit_list=np.unique(magic_yinit_list)

    #convert yinit_list to cinit_list for cthetadeg=60, since that's the magic cone angle that yinit concerns, then append to list.
    magic_cinit_list=np.append(magic_cinit_list,magic_yinit_list*(60/180*np.pi))

    #add additional cinit values to increase resolution
    magic_cinit_list=np.append(magic_cinit_list,np.arange(cinit_min,cinit_max,0.5))
    
    #filter based on cinit range
    magic_cinit_list=magic_cinit_list[np.where((magic_cinit_list<=cinit_max)&(magic_cinit_list>=cinit_min))]
    magic_cinit_list=np.unique(magic_cinit_list)

    if 5*np.sqrt(3) in magic_cinit_list: #ithetadeg=60 magic cylinder case
        print('#ithetadeg=60 magic cylinder case included')
    if 5 in magic_cinit_list: #ithetadeg=0 magic cylinder case
        print('#ithetadeg=0 magic cylinder case included')
    if 7*np.sqrt(3)/2*(60/180*np.pi) in magic_cinit_list: #ithetadeg=0 magic cone case
        print('#ithetadeg=0 magic cone case included')
    if 7*(60/180*np.pi) in magic_cinit_list: #ithetadeg=60 magic cone case
        print('#ithetadeg=60 magic cone case included')
    return magic_cinit_list

Modify the following code block with desired parameters.

In [ ]:
def generate_run_dir_list():
    cthetadeg_list=np.linspace(0,30,7) #cylinders and cones
    cinit_list=np.linspace(0,12,49)
    # cinit_list=magic_cinit_list() #generate values that contain commensurate values
    ithetadeg_list=np.linspace(59,60,1)
    trials_list=np.arange(0,100,1)
    Nmax_list=np.linspace(1000,1000,1)

    print('\ncthetadeg \n',cthetadeg_list)
    print('cinit \n',cinit_list)
    print('ithetadeg \n',ithetadeg_list)
    print('trials \n',trials_list)
    print('Nmax \n',Nmax_list)

    run_dir_list=[]
    for trial in trials_list:
        for cthetadeg in cthetadeg_list:
            for cinit in cinit_list: #circumference to initialize seed at, equivalent to arc length. cinit=s=r*ctheta=yinit*ctheta
                for ithetadeg in ithetadeg_list:
                    for Nmax in Nmax_list: #won't be max packing for cylinders but should be sufficient >1deg
                        run_dir = ['%s-%s_%s-%s_%s-%s_%s-%s_%s-%s' % \
                                  ('cthetadeg', str(cthetadeg), 'cinit', str(cinit),'ithetadeg', str(ithetadeg),'Nmax', str(Nmax),'trial', str(trial))] #_%s, str(rep)
                        run_dir_list+=run_dir
    run_dir_list=np.array(run_dir_list)
    filename='run_dir_list_custom_'+str(np.min(trials_list))+'-'+str(np.max(trials_list))#+'_cylinders0-5cinit'
    np.save(filename,run_dir_list)
    print('runtime: ~'+str(int(((len(run_dir_list)*0.5)/60)/24))+' days')
    print(filename)
    return run_dir_list
run_dir_list=generate_run_dir_list()

---
2. Edit existing run_dir_list.npy

In [ ]:
def edit_run_dir_list():
    edit_existing=np.load('/users/jessicasun/Documents/GitHub/cone-disk-packings/simulation/simulation_inputs/run_dir_list_022723_0-99.npy')

    remaining_trials=np.array([14])
    remaining_cthetadeg=np.array([5.0])
    remaining_cinit=np.array([5.0])
    remaining_ithetadeg=np.array([10.0])
    
    run_dir_list_remaining=[]
    for i in edit_existing:
        trial_num=int(i.split('-')[-1])
        cthetadeg=float(i.split('_')[0].split('-')[1])
        cinit=float(i.split('_')[1].split('-')[1])
        ithetadeg=float(i.split('_')[2].split('-')[1])        
        if trial_num in remaining_trials:
            if cthetadeg in remaining_cthetadeg:
                if cinit in remaining_cinit:
                    if ithetadeg in remaining_ithetadeg:
                        run_dir_list_remaining+=[i]    
    np.save('run_dir_list_remaining',run_dir_list_remaining)            
    return run_dir_list_remaining
run_dir_list_remaining=edit_run_dir_list()

---
3. Generate run_dir_list.npy for commensurate cylinders (using parastichy number conventions) and cylinder_params_df.pkl (not used as simulation input, but needed for analysis)

In [ ]:
def get_cylinder_cinit(m,n,a): #find commensurate cylinder cinit for a given initial orientation
    cinit=a*np.sqrt(m**2+n**2-(m*n))
    return cinit
def get_cylinder_ithetadeg(m,n):
    mn_ratio=m/n
    phi_dbeller=np.arctan((2/np.sqrt(3))*((mn_ratio)-(1/2))) #phi based on DA Beller, DR Nelson convention (angle made with vertical axis)
    phi_dbeller_deg=phi_dbeller*180/np.pi
    ithetadeg=(90-phi_dbeller_deg) #theta based on our convention, angle made with circumference
    return phi_dbeller_deg,ithetadeg
def get_cylinder_params(m,n,a): #characterize in terms of parastichy numbers
    phi_dbeller_deg,ithetadeg=get_cylinder_ithetadeg(m,n)
    cinit=get_cylinder_cinit(m,n,a)
    mn_ratio=m/n
    cylinder_params_df=pd.DataFrame([{'m':m,'n':n,'mn_ratio':mn_ratio,'phi_dbeller_deg':phi_dbeller_deg,'ithetadeg':ithetadeg,'a':a,'cinit':cinit}])
    return cylinder_params_df

In [ ]:
a_list=np.arange(1,11,1)
cylinder_params_df_list=[]
mn_list=[(1,1),(2,1),(3,2),(4,3),(5,4)]
for mn in mn_list:
    m,n=mn[0],mn[1]
    for a in a_list:
        cylinder_params_df=get_cylinder_params(m,n,a)
        cylinder_params_df_list+=[cylinder_params_df]    
cylinder_params_df=pd.concat(cylinder_params_df_list)
cylinder_params_df=cylinder_params_df[cylinder_params_df['cinit']<=12] #from experience i know that simulations break down at high circumferences due to numerical error

In [ ]:
cylinder_params_df

In [ ]:
cylinder_params_df.to_pickle('cylinder_params_df.pkl')

In [ ]:
def generate_cylinder_run_dir_list(cylinder_params_df):
    cthetadeg_list=[0] #cylinders only
    cinit_list=np.unique(np.array(cylinder_params_df['cinit']))

    ithetadeg_list=np.unique(np.array(cylinder_params_df['ithetadeg']))
    trials_list=np.arange(0,3,1)
    Nmax_list=np.linspace(1000,1000,1)

    print('\ncthetadeg \n',cthetadeg_list)
    print('cinit \n',cinit_list)
    print('ithetadeg \n',ithetadeg_list)
    print('trials \n',trials_list)
    print('Nmax \n',Nmax_list)

    run_dir_list=[]
    for trial in trials_list:
        for cthetadeg in cthetadeg_list:
            for cinit in cinit_list: #circumference to initialize seed at, equivalent to arc length. cinit=s=r*ctheta=yinit*ctheta
                for ithetadeg in ithetadeg_list:
                    for Nmax in Nmax_list: #won't be max packing for cylinders but should be sufficient >1deg
                        run_dir = ['%s-%s_%s-%s_%s-%s_%s-%s_%s-%s' % \
                                  ('cthetadeg', str(cthetadeg), 'cinit', str(cinit),'ithetadeg', str(ithetadeg),'Nmax', str(Nmax),'trial', str(trial))] #_%s, str(rep)
                        run_dir_list+=run_dir
    run_dir_list=np.array(run_dir_list)
    filename='custom_'+str(np.min(trials_list))+'-'+str(np.max(trials_list))+'cinit0-12'#+'_cylinders0-5cinit'
    np.save(filename,run_dir_list)
    print('runtime: ~'+str(int(((len(run_dir_list)*0.5)/60)/24))+' days')
    print(filename)
    return run_dir_list
run_dir_list=generate_cylinder_run_dir_list(cylinder_params_df)